In [ ]:
#=================================================
# Copyright (c) 2025, Security Pattern
# All rights reserved.
#
#    This file is part of: Analysis of ountermeasures used to protect the Ascon algorithm.
#
#    SPDX-License-Identifier: MIT 
#=================================================

import numpy as np
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.palettes import Category10 as palette
colors = palette[10]
import math
import glob
import ascon128_dom_hw_v1_random as ascon

# Parameters

In [ ]:
# Ascon parameters
rate = 8
a = 12
b = 6
k = 16 * 8
len_v_rand = 5*(2*a+b*(math.ceil((16+1)/rate)+math.ceil(((16+1)/rate)-1)))
# Traces
n_traces = 50000 # Total number of acquired traces
NS = 1000 # Number of traces for each file

# Functional Functions

In [4]:
# Load of traces and corresponding data
def load_data_traces(folder, n_traces):   
    n_files = math.ceil(n_traces/NS)
    for file_data in glob.glob(folder+"/data*"):
        s = file_data[len(file_data)-19:]
        file_traces = glob.glob(folder+"/traces*"+s)
        if(n_files == 0): break
        else: n_files = n_files - 1
        if n_files + 1 == math.ceil(n_traces/NS):
            data = np.load(file_data)
            traces = np.load(file_traces[0])
        else: 
            data = np.concatenate((data, np.load(file_data)))
            traces = np.concatenate((traces, np.load(file_traces[0])))
    traces = np.array(traces)
    return(traces, data)

# TVLA computation
def tvla_1_Order(traces_fixed, traces_random):
    nb_of_traces_fixed = len(traces_fixed)
    nb_of_traces_random = len(traces_random)
    M1_r = np.mean(traces_random, axis=0)
    CM2_r = np.var(traces_random, axis=0)
    M1_f = np.mean(traces_fixed, axis=0)
    CM2_f = np.var(traces_fixed, axis=0)
    dist = M1_f - M1_r
    den = np.sqrt((CM2_f/nb_of_traces_fixed) + (CM2_r/nb_of_traces_random))
    den = [pow(10,10) if x == 0 else x for x in den]
    return dist/den

# Add the grid that allows to recognize the cycles in the initialization phase
def add_mean_grid_fig_initialization(p, x_min, x_max):
    rounds = [144 + 16 * x for x in list(range(13))]
    mid_rounds = [152 + 16 * x for x in list(range(12))]
    for i in range(13):
        p.line(rounds[i], [x_min, x_max], line_color=colors[3]) 
    for i in range(12):
        p.line(mid_rounds[i], [x_min,x_max], line_color=colors[3], line_dash="dashed")

# Xor two bytes strings
def byte_xor(ba1, ba2):
    return bytes([_a ^ _b for _a, _b in zip(ba1, ba2)])

def to_bytes(l):
    return bytes(bytearray(l))

def int_to_bytes(integer, nbytes):
    return to_bytes([(integer >> ((nbytes - 1 - i) * 8)) % 256 for i in range(nbytes)])

# Compute the logical state from the states of the two shares
# logical_state is a list of lists
def logical_state_computation(S1, S2, n_traces):
    logical_state = []
    for i in range(0, n_traces):
        S = [0, 0, 0, 0, 0]
        for j in range(5): S[j] = S2[i][j] ^ S1[i][j]
        logical_state = logical_state + [S]
    return logical_state

# function that computes the hw of bits of the state specified by "bits"
# "bits" need to be a multiple of 8
# bits are taken from the end to the beginning of the state
def hw_bytes(S, bits = 320):
    h = 0
    if (bits % 8 != 0):
        print("Bits must be a multiple of 8.")
        return 0
    else:
        for i in range(4,-1,-1):
            S_block_bytes = int_to_bytes(S[i], 8)
            if(bits >= 64): 
                h = h + sum([S_block_bytes[j].bit_count() for j in range(8)])
                bits = bits - 64
                if bits == 0: return h
            else: 
                h = h + sum([S_block_bytes[j].bit_count() for j in range(int(bits/8),8)])
                return h

# S is the state, that is a list of 5 integers
# bit_mask is a list of 5 bitstrings, which explicit with bits we want to consider for the hamming weight computation
def hw_with_mask(S, bit_mask):
    HW = 0
    for i in range(0,5):
        a = S[i] & int(bit_mask[i],2)
        HW += a.bit_count()
    return HW

# Compute the leakage model
# input1 and input2 are states; if the type id HW then input2 is not needed
def leakage_model(type, bit_mask, input1, input2=-1):
    match type:
        case 'HW':
            return hw_with_mask(input1, bit_mask)
        case 'HD':
            if input2 == -1:
                print("leakage model HD needs a second input")
                return 0
            Sxor = [input1[j] ^ input2[j] for j in range(5)]
            return hw_with_mask(Sxor, bit_mask)

# Compute the hypothesis matrix
def compute_H_matrix(state1, bit_mask, type, n_traces, state2 = n_traces*[-1]):
    H = []
    for i in range(0,n_traces):
        H = H + [leakage_model(type, bit_mask, state1[i], state2[i])]
    return H

# correlation function between the hypothesis matrix and the traces matrix
def my_correlation(H_matrix, T_matrix):
    corr1 = []
    for i in range(0,len(T_matrix)):
        corr1 = corr1 + [np.corrcoef(H_matrix, T_matrix[i])[0][1]]
    return corr1

# Ascon with DOM functions

In [ ]:
# return 2 lists (of lenght n_traces) of lists (the shares states, i.e. 5 lane of 64-bits integers)
# state_share1 = first share of the initial state, state_share2 = second share of the initial state
def initial_states_computation(IV1, IV2, Key1, Key2, n_traces):

    state_share1 = []
    state_share2 = []
    
    for i in range(0,n_traces):
        
        s1 = [0, 0, 0, 0, 0]
        iv_zero_key_nonce1 = ascon.to_bytes(8*[0]) + Key1[i] + IV1[i]
        s1[0], s1[1], s1[2], s1[3], s1[4] = ascon.bytes_to_state(iv_zero_key_nonce1)
        state_share1 = state_share1 + [s1]
        s2 = [0, 0, 0, 0, 0]
        iv_zero_key_nonce2 = ascon.to_bytes([k, rate * 8, a, b] + (20-len(Key2[i]))*[0]) + Key2[i] + IV2[i]
        s2[0], s2[1], s2[2], s2[3], s2[4] = ascon.bytes_to_state(iv_zero_key_nonce2)
        state_share2 = state_share2 + [s2]

    return state_share1, state_share2


# round in the permutation -> first part (until the application of DOM)
def round_firstpart_phase(r, S1, S2, n_round, v_rand, n_traces):

    T1 = []
    T2 = []
    rand_idx = 5*n_round
        
    for i in range(0, n_traces):

        # --- add round constants ---
        S1[i][2] ^= (0xf0 - r*0x10 + r*0x1) # constant is added only to one share

        # --- substitution layer ---
        S1[i][0] = S1[i][0] ^ S1[i][4]  
        S1[i][2] = S1[i][2] ^ S1[i][1] 
        S1[i][4] = S1[i][4] ^ S1[i][3] 

        S2[i][0] = S2[i][0] ^ S2[i][4] 
        S2[i][2] = S2[i][2] ^ S2[i][1]
        S2[i][4] = S2[i][4] ^ S2[i][3] 
        
        T = [ascon.masked_and(S1[i][j], S1[i][(j+1)%5], S2[i][j], S2[i][(j+1)%5], v_rand[i][rand_idx+j]) for j in range(5)]
        T1 = T1 + [[T[j][0] for j in range(5)]]
        T2 = T2 + [[T[j][1] for j in range(5)]]

    return T1, T2


# round in the permutation -> second part (after the application of DOM)
def round_secondpart_phase(S1, S2, n_traces, T1, T2):

    for i in range(0,n_traces):

        for j in range(5):
            S1[i][j] ^= T1[i][(j+1)%5]
        S1[i][1] ^= S1[i][0]
        S1[i][0] ^= S1[i][4]
        S1[i][3] ^= S1[i][2]
        S1[i][2] ^= 0XFFFFFFFFFFFFFFFF

        for j in range(5):
            S2[i][j] ^= T2[i][(j+1)%5]
        S2[i][1] ^= S2[i][0]
        S2[i][0] ^= S2[i][4]
        S2[i][3] ^= S2[i][2]

        # --- linear diffusion layer ---
        S1[i][0] ^= ascon.rotr(S1[i][0], 19) ^ ascon.rotr(S1[i][0], 28)
        S1[i][1] ^= ascon.rotr(S1[i][1], 61) ^ ascon.rotr(S1[i][1], 39)
        S1[i][2] ^= ascon.rotr(S1[i][2],  1) ^ ascon.rotr(S1[i][2],  6)
        S1[i][3] ^= ascon.rotr(S1[i][3], 10) ^ ascon.rotr(S1[i][3], 17)
        S1[i][4] ^= ascon.rotr(S1[i][4],  7) ^ ascon.rotr(S1[i][4], 41)

        S2[i][0] ^= ascon.rotr(S2[i][0], 19) ^ ascon.rotr(S2[i][0], 28)
        S2[i][1] ^= ascon.rotr(S2[i][1], 61) ^ ascon.rotr(S2[i][1], 39)
        S2[i][2] ^= ascon.rotr(S2[i][2],  1) ^ ascon.rotr(S2[i][2],  6)
        S2[i][3] ^= ascon.rotr(S2[i][3], 10) ^ ascon.rotr(S2[i][3], 17)
        S2[i][4] ^= ascon.rotr(S2[i][4],  7) ^ ascon.rotr(S2[i][4], 41)

# Data computation function and data check function

In [ ]:
# Computation of the list of data
# rnd_mode is 0 if all the randoms are zeros, rnd_mode is 1 viceversa
def data_computation(data, rnd_mode, n_traces): 

    IV1, IV2 = [], []
    Key1, Key2 = [], []
    AD1, AD2 = [], []
    PT1, PT2 = [], []
    TAG = []
    Z = []

    for i in range(0,n_traces):

        Key1 = Key1 + [data[i][0]]
        Key2 = Key2 + [data[i][1]]
        IV1 = IV1 + [data[i][2]]
        IV2 = IV2 + [data[i][3]]
        AD1 = AD1 + [data[i][4]]
        AD2 = AD2 + [data[i][5]]
        PT1 = PT1 + [data[i][6]]
        PT2 = PT2 + [data[i][7]]
        TAG = TAG + [data[i][40-16:40]]
        # Random
        if rnd_mode == 0:
            Z = Z + [[0X0000000000000000]*len_v_rand]
        else:
            rnd = data[i][40:]
            rnd1 = []
            rnd2 = bytearray(int(b.decode()) for b in rnd)
            for i in range(int(480/40)):
                rnd1 = rnd1 + ascon.bytes_to_state(rnd2[i*40:(i+1)*40])
            for i in range(int(210/5)):
                rnd1 = rnd1 + ascon.bytes_to_state(bytearray(210))
            Z = Z + [rnd1]

    return IV1, IV2, Key1, Key2, AD1, AD2, PT1, PT2, TAG, Z


# CHECK THE DATA from acquisitions
def check_traces(data, traces, n_traces, iv1, iv2, k1, k2, ad1, ad2, pt1, pt2, tag, z):
    data_cleaned = []
    traces_cleaned = []
    n = 0
    K1, K2, IV1, IV2, AD1, AD2, PT1, PT2, TAG, Z = [], [], [], [], [], [], [], [], [], []
    for i in range(0,n_traces):
        if (all([len(k1[i])==16, len(k2[i])==16, len(iv1[i])==16, len(iv2[i])==16, len(pt1[i])==16, len(pt2[i])==16])):
            ct1_tg1, ct2_tg2 = ascon.ascon_encrypt_shares(k2[i], k1[i], iv2[i], iv1[i], ad2[i], ad1[i], pt2[i], pt1[i], a, b, rate, z[i])
            tg1 = ct1_tg1[(int(len(ct1_tg1)/2)):len(ct1_tg1)]
            tg2 = ct2_tg2[(int(len(ct2_tg2)/2)):len(ct2_tg2)]
            tagg = bytearray(int(b.decode()) for b in tag[i])
            if (tagg == byte_xor(tg1,tg2)):
                K1 = K1 + [bytearray(k1[i])]
                K2 = K2 + [bytearray(k2[i])]
                IV1 = IV1 + [bytearray(iv1[i])]
                IV2 = IV2 + [bytearray(iv2[i])]
                AD1 = AD1 + [bytearray(ad1[i])]
                AD2 = AD2 + [bytearray(ad2[i])]
                PT1 = PT1 + [bytearray(pt1[i])]
                PT2 = PT2 + [bytearray(pt2[i])]
                TAG = TAG + [bytearray(tag[i])]
                Z = Z + [z[i]]
                data_cleaned = data_cleaned + [data[i]]
                traces_cleaned = traces_cleaned + [traces[i]]
                n = n +1

    return K1, K2, IV1, IV2, AD1, AD2, PT1, PT2, TAG, Z, n, data_cleaned, traces_cleaned


# Ascon with DOM and random

## Load traces and check

In [20]:
pname_random = "./acquisition/t_test_50000fix_50000rnd_DOM_allsharesvarying/Random"
pname_fixed  = "./acquisition/t_test_50000fix_50000rnd_DOM_allsharesvarying/Fixed"

n_traces = 50000 # Total number of acquired traces

traces_rnd, data_rnd = load_data_traces(pname_random, n_traces) 
traces_fix, data_fix = load_data_traces(pname_fixed, n_traces) 

In [46]:
iv1, iv2, k1, k2, ad1, ad2, pt1, pt2, tag, z = data_computation(data_rnd, 1, int(n_traces/2))
K1, K2, IV1, IV2, AD1, AD2, PT1, PT2, TAG, Z, n_traces, data_cleaned, T = check_traces(data_rnd, traces_rnd, int(n_traces/2), iv1, iv2, k1, k2, ad1, ad2, pt1, pt2, tag, z)
print(n_traces)

24315


## Mean, variance, TVLA

In [21]:
# MEAN

mean_fix = np.mean(traces_fix,axis=0)
mean_rnd = np.mean(traces_rnd,axis=0)

output_notebook()
p = figure()
xrange = list(range(len(mean_fix)))
p.line(xrange, mean_fix, legend_label="FIXED", line_color="blue")
p.line(xrange, mean_rnd, legend_label="RANDOM", line_color="red")
#add_mean_grid_fig_initialization(p, -0.6, 0.4)
show(p)

Loading BokehJS ...

In [ ]:
#VARIANCE

var_fix = np.var(traces_fix,axis=0)
var_rnd = np.var(traces_rnd,axis=0)

output_notebook()
p = figure()
xrange = list(range(len(var_fix)))
p.line(xrange, var_fix, legend_label= "FIXED", line_color="blue")
p.line(xrange, var_rnd, legend_label="RANDOM", line_color="red")
show(p)

Loading BokehJS ...

In [19]:
# TVLA

ttest = tvla_1_Order(traces_fix, traces_rnd)

output_notebook()
p = figure()
xrange = list(range(len(ttest)))
p.line(xrange, ttest, line_color="green")
p.line(xrange,[4.5]*len(ttest),line_color="red")
p.line(xrange,[-4.5]*len(ttest),line_color="red")
# Threshold
p.y_range.start = -160
p.y_range.end = 220
show(p)

Loading BokehJS ...

# Ascon with DOM and random at zero

## Load traces and check

In [22]:
pname_random = "./acquisition/t_test_50000fix_50000rnd_DOM_allsharesvarying_random0/Random"
pname_fixed  = "./acquisition/t_test_50000fix_50000rnd_DOM_allsharesvarying_random0/Fixed"
n_traces = 50000 # Total number of acquired traces

traces_rnd_rnd0, data_rnd_rnd0 = load_data_traces(pname_random, n_traces) 
traces_fix_rnd0, data_fix_rnd0 = load_data_traces(pname_fixed, n_traces) 

In [15]:
iv1, iv2, k1, k2, ad1, ad2, pt1, pt2, tag, z = data_computation(data_rnd_rnd0, 0, int(n_traces/2))
K1, K2, IV1, IV2, AD1, AD2, PT1, PT2, TAG, Z, n_traces, data_cleaned, T = check_traces(data_rnd_rnd0, traces_rnd_rnd0, int(n_traces/2), iv1, iv2, k1, k2, ad1, ad2, pt1, pt2, tag, z)
print(n_traces)

24325


## Mean, variance, TVLA

In [23]:
# MEAN

mean_fix = np.mean(traces_fix_rnd0,axis=0)
mean_rnd = np.mean(traces_rnd_rnd0,axis=0)

output_notebook()
p = figure()
xrange = list(range(len(mean_fix)))
p.line(xrange, mean_fix, legend_label="FIXED", line_color="blue")
p.line(xrange, mean_rnd, legend_label="RANDOM", line_color="red")
#add_mean_grid_fig_initialization(p, -0.6, 0.4)
show(p)

Loading BokehJS ...

In [ ]:
#VARIANCE

var_fix = np.var(traces_fix_rnd0,axis=0)
var_rnd = np.var(traces_rnd_rnd0,axis=0)

output_notebook()
p = figure()
xrange = list(range(len(var_fix)))
p.line(xrange, var_fix, legend_label= "FIXED", line_color="blue")
p.line(xrange, var_rnd, legend_label="RANDOM", line_color="red")
show(p)

Loading BokehJS ...

In [ ]:
# TVLA

ttest = tvla_1_Order(traces_fix_rnd0, traces_rnd_rnd0)

output_notebook()
p = figure()
xrange = list(range(len(ttest)))
p.line(xrange, ttest, line_color="green")
p.line(xrange,[4.5]*len(ttest),line_color="red")
p.line(xrange,[-4.5]*len(ttest),line_color="red")
# Threshold
p.y_range.start = -160
p.y_range.end = 220
show(p)

Loading BokehJS ...

# Correlation between the values of the traces at each sample and a leakage prevision

## Hamming Weight of the logical state of 32-bits words of the key

In [19]:
# computation of the initial state
S1, S2 = initial_states_computation(IV2, IV1, K2, K1, n_traces)
S = logical_state_computation(S1, S2, n_traces)

# correlation computation

bit_mask_S_1_0 = ['0b0000000000000000000000000000000000000000000000000000000000000000',
                  '0b0000000000000000000000000000000011111111111111111111111111111111',
                  '0b0000000000000000000000000000000000000000000000000000000000000000',
                  '0b0000000000000000000000000000000000000000000000000000000000000000',
                  '0b0000000000000000000000000000000000000000000000000000000000000000']
bit_mask_S_1_1 = ['0b0000000000000000000000000000000000000000000000000000000000000000',
                  '0b1111111111111111111111111111111100000000000000000000000000000000',
                  '0b0000000000000000000000000000000000000000000000000000000000000000',
                  '0b0000000000000000000000000000000000000000000000000000000000000000',
                  '0b0000000000000000000000000000000000000000000000000000000000000000']
bit_mask_S_2_0 = ['0b0000000000000000000000000000000000000000000000000000000000000000',
                  '0b0000000000000000000000000000000000000000000000000000000000000000',
                  '0b0000000000000000000000000000000011111111111111111111111111111111',
                  '0b0000000000000000000000000000000000000000000000000000000000000000',
                  '0b0000000000000000000000000000000000000000000000000000000000000000']
bit_mask_S_2_1 = ['0b0000000000000000000000000000000000000000000000000000000000000000',
                  '0b0000000000000000000000000000000000000000000000000000000000000000',
                  '0b1111111111111111111111111111111100000000000000000000000000000000',
                  '0b0000000000000000000000000000000000000000000000000000000000000000',
                  '0b0000000000000000000000000000000000000000000000000000000000000000']

H_S_1_0 = compute_H_matrix(S, bit_mask_S_1_0, 'HW', n_traces) 
H_S_1_1 = compute_H_matrix(S, bit_mask_S_1_1, 'HW', n_traces) 
H_S_2_0 = compute_H_matrix(S, bit_mask_S_2_0, 'HW', n_traces) 
H_S_2_1 = compute_H_matrix(S, bit_mask_S_2_1, 'HW', n_traces) 

In [21]:
t = np.transpose(T)
corr_S_1_0 = my_correlation(H_S_1_0, t)
corr_S_1_1 = my_correlation(H_S_1_1, t)
corr_S_2_0 = my_correlation(H_S_2_0, t)
corr_S_2_1 = my_correlation(H_S_2_1, t)

output_notebook()
p = figure(title="Correlation of the traces with HW of 32 bits of the key, logical state,\n before the start of the initialization")
p.xaxis.axis_label = "Samples"
p.yaxis.axis_label = "Correlation"
xlen =  len(corr_S_1_0)
xrange = list(range(xlen))
p.line(xrange, corr_S_1_0, line_color=colors[0], legend_label="Word of the key [0:32]")   
p.line(xrange, corr_S_1_1, line_color=colors[1], legend_label="Word of the key [32:64]")   
p.line(xrange, corr_S_2_0, line_color=colors[2], legend_label="Word of the key [64:96]")   
p.line(xrange, corr_S_2_1, line_color=colors[3], legend_label="Word of the key [96:128]")   
p.legend.label_text_font_size = "8pt"
p.y_range.start = -0.65
p.y_range.end = 0.3

show(p)

Loading BokehJS ...

## Hamming Distance of the logical state of 32-bits words of the key

In [22]:
# computation of the initial state
S1, S2 = initial_states_computation(IV2, IV1, K2, K1, n_traces)
S = logical_state_computation(S1, S2, n_traces)

block_S_1_0 =  []
block_S_1_1 =  []
block_S_2_0 =  []
block_S_2_1 =  []
# block_Sn_m_q means share n, lane m, q=0 is 32 less significant bits q=1 is 32 most significant bits
for i in range(0,n_traces):
    block_S_1_0 = block_S_1_0 + [[0, 0, 0, 0, S[i][1]>>32]]
    block_S_1_1 = block_S_1_1 + [[0, 0, 0, 0, S[i][1]]]
    block_S_2_0 = block_S_2_0 + [[0, 0, 0, 0, S[i][2]>>32]]
    block_S_2_1 = block_S_2_1 + [[0, 0, 0, 0, S[i][2]]]

# correlation computation

bit_mask = ['0b0000000000000000000000000000000000000000000000000000000000000000',
            '0b0000000000000000000000000000000000000000000000000000000000000000',
            '0b0000000000000000000000000000000000000000000000000000000000000000',
            '0b0000000000000000000000000000000000000000000000000000000000000000',
            '0b0000000000000000000000000000000011111111111111111111111111111111']

H1 = compute_H_matrix(block_S_1_0, bit_mask, 'HD', n_traces, block_S_1_1) 
H2 = compute_H_matrix(block_S_1_1, bit_mask, 'HD', n_traces, block_S_2_0) 
H3 = compute_H_matrix(block_S_2_0, bit_mask, 'HD', n_traces, block_S_2_1) 
H4 = compute_H_matrix(block_S_2_1, bit_mask, 'HD', n_traces, block_S_1_0) 

In [23]:
t = np.transpose(T)
corr_S_1_0 = my_correlation(H1, t)
corr_S_1_1 = my_correlation(H2, t)
corr_S_2_0 = my_correlation(H3, t)
corr_S_2_1 = my_correlation(H4, t)

output_notebook()
p = figure(title="Correlation of the traces with HD of 32 bits of the key, logical state,\n before the start of the initialization")
p.xaxis.axis_label = "Samples"
p.yaxis.axis_label = "Correlation"
xlen =  len(corr_S_1_0)
xrange = list(range(xlen))
p.line(xrange, corr_S_1_0, line_color=colors[0], legend_label="Word of the key [0:32]")   
p.line(xrange, corr_S_1_1, line_color=colors[1], legend_label="Word of the key [32:64]")   
p.line(xrange, corr_S_2_0, line_color=colors[2], legend_label="Word of the key [64:96]")   
p.line(xrange, corr_S_2_1, line_color=colors[3], legend_label="Word of the key [96:128]")   
p.legend.label_text_font_size = "8pt"
p.y_range.start = -0.65
p.y_range.end = 0.3

show(p)

Loading BokehJS ...

## HW of 32-bits words of the first shares of the key, concatenated with the corresponding 32-bits word of the second share of the key

In [24]:
# computation of the initial state
#S1, S2 = initial_states_computation(IV1, IV2, K1, K2, n_traces)
S1, S2 = initial_states_computation(IV2, IV1, K2, K1, n_traces)
S = logical_state_computation(S1, S2, n_traces)

# correlation computation

bit_mask_S_1_0 = ['0b0000000000000000000000000000000000000000000000000000000000000000',
                  '0b0000000000000000000000000000000011111111111111111111111111111111',
                  '0b0000000000000000000000000000000000000000000000000000000000000000',
                  '0b0000000000000000000000000000000000000000000000000000000000000000',
                  '0b0000000000000000000000000000000000000000000000000000000000000000']
bit_mask_S_1_1 = ['0b0000000000000000000000000000000000000000000000000000000000000000',
                  '0b1111111111111111111111111111111100000000000000000000000000000000',
                  '0b0000000000000000000000000000000000000000000000000000000000000000',
                  '0b0000000000000000000000000000000000000000000000000000000000000000',
                  '0b0000000000000000000000000000000000000000000000000000000000000000']
bit_mask_S_2_0 = ['0b0000000000000000000000000000000000000000000000000000000000000000',
                  '0b0000000000000000000000000000000000000000000000000000000000000000',
                  '0b0000000000000000000000000000000011111111111111111111111111111111',
                  '0b0000000000000000000000000000000000000000000000000000000000000000',
                  '0b0000000000000000000000000000000000000000000000000000000000000000']
bit_mask_S_2_1 = ['0b0000000000000000000000000000000000000000000000000000000000000000',
                  '0b0000000000000000000000000000000000000000000000000000000000000000',
                  '0b1111111111111111111111111111111100000000000000000000000000000000',
                  '0b0000000000000000000000000000000000000000000000000000000000000000',
                  '0b0000000000000000000000000000000000000000000000000000000000000000']

H_S1_1_0 = compute_H_matrix(S1, bit_mask_S_1_0, 'HW', n_traces) 
H_S2_1_0 = compute_H_matrix(S2, bit_mask_S_1_0, 'HW', n_traces) 
H_conc_1_0 = [a + b for a, b in zip(H_S1_1_0, H_S2_1_0)]

H_S1_1_1 = compute_H_matrix(S1, bit_mask_S_1_1, 'HW', n_traces) 
H_S2_1_1 = compute_H_matrix(S2, bit_mask_S_1_1, 'HW', n_traces) 
H_conc_1_1 = [a + b for a, b in zip(H_S1_1_1, H_S2_1_1)]

H_S1_2_0 = compute_H_matrix(S1, bit_mask_S_2_0, 'HW', n_traces) 
H_S2_2_0 = compute_H_matrix(S2, bit_mask_S_2_0, 'HW', n_traces) 
H_conc_2_0 = [a + b for a, b in zip(H_S1_2_0, H_S2_2_0)]

H_S1_2_1 = compute_H_matrix(S1, bit_mask_S_2_1, 'HW', n_traces) 
H_S2_2_1 = compute_H_matrix(S2, bit_mask_S_2_1, 'HW', n_traces) 
H_conc_2_1 = [a + b for a, b in zip(H_S1_2_1, H_S2_2_1)]

In [27]:
t = np.transpose(T)
corr_S_1_0 = my_correlation(H_conc_1_0, t)
corr_S_1_1 = my_correlation(H_conc_1_1, t)
corr_S_2_0 = my_correlation(H_conc_2_0, t)
corr_S_2_1 = my_correlation(H_conc_2_1, t)

output_notebook()
p = figure(title="Correlation of the traces with HW of 32 bits of the key, share 1 concatenated to share 2,\n before the start of the initialization")
p.xaxis.axis_label = "Samples"
p.yaxis.axis_label = "Correlation"
xlen =  len(corr_S_1_0)
xrange = list(range(xlen))
p.line(xrange, corr_S_1_0, line_color=colors[0], legend_label="Word of the key [0:32]")   
p.line(xrange, corr_S_1_1, line_color=colors[1], legend_label="Word of the key [32:64]")   
p.line(xrange, corr_S_2_0, line_color=colors[2], legend_label="Word of the key [64:96]")   
p.line(xrange, corr_S_2_1, line_color=colors[3], legend_label="Word of the key [96:128]")   
p.legend.label_text_font_size = "8pt"
p.y_range.start = -0.65
p.y_range.end = 0.3

show(p)

Loading BokehJS ...

## HD or HW of all the bits of S1||S2, first round

In [28]:
# computation of the initial state
S1, S2 = initial_states_computation(IV2, IV1, K2, K1, n_traces)
# INITIAL STATE COMPOSITION: [IV' (64 bits) | Key (128 bits) | IV (128 bits)]
S1_b, S2_b = initial_states_computation(IV2, IV1, K2, K1, n_traces)

# computation of the state after the first part of the permutation
n_round =0
T1, T2 = round_firstpart_phase(0,S1,S2,n_round,Z,n_traces)

# computation of the state after the second part of the permutation
round_secondpart_phase(S1,S2,n_traces,T1,T2)

# correlation computation

bit_mask_whole = ['0b1111111111111111111111111111111111111111111111111111111111111111',
                  '0b1111111111111111111111111111111111111111111111111111111111111111',
                  '0b1111111111111111111111111111111111111111111111111111111111111111',
                  '0b1111111111111111111111111111111111111111111111111111111111111111',
                  '0b1111111111111111111111111111111111111111111111111111111111111111']

H_S1_ab_whole = compute_H_matrix(S1_b, bit_mask_whole, 'HD',n_traces, S1)
H_S2_ab_whole = compute_H_matrix(S2_b, bit_mask_whole, 'HD',n_traces, S2) 
H_conc_ab_whole = [a + b for a, b in zip(H_S1_ab_whole, H_S2_ab_whole)]

H_S1_input = compute_H_matrix(S1_b, bit_mask_whole, 'HW', n_traces) 
H_S2_input = compute_H_matrix(S2_b, bit_mask_whole, 'HW', n_traces) 
H_conc_input = [a + b for a, b in zip(H_S1_input, H_S2_input)]

H_S1_output = compute_H_matrix(S1, bit_mask_whole, 'HW', n_traces) 
H_S2_output = compute_H_matrix(S2, bit_mask_whole, 'HW', n_traces) 
H_conc_output = [a + b for a, b in zip(H_S1_output, H_S2_output)]

In [35]:
t = np.transpose(T)
corr_HD = my_correlation(H_conc_ab_whole, t)
corr_HWinput = my_correlation(H_conc_input, t)
corr_HWoutput = my_correlation(H_conc_output, t)

output_notebook()
p = figure(title="Correlation of the traces with HD or HW of all the bits of S1||S2, first round")
p.xaxis.axis_label = "Samples"
p.yaxis.axis_label = "Correlation"
xlen =  len(corr_HD)
xrange = list(range(xlen))
p.line(xrange, corr_HD, line_color=colors[0], legend_label="HD input/output first round")   
p.line(xrange, corr_HWinput, line_color=colors[1], legend_label="HW input first round")  
p.line(xrange, corr_HWoutput, line_color=colors[2], legend_label="HW ouptut first round") 
add_mean_grid_fig_initialization(p, -0.6, 0.30)

show(p)

Loading BokehJS ...

## HD or HW of all the bits of S1||S2, second round

In [36]:
# computation of the initial state
S1, S2 = initial_states_computation(IV2, IV1, K2, K1, n_traces)
SS1, SS2 = initial_states_computation(IV2, IV1, K2, K1, n_traces)
# INITIAL STATE COMPOSITION: [IV' (64 bits) | Key (128 bits) | IV (128 bits)]

# FIRST ROUND
n_round =0
T1, T2 = round_firstpart_phase(0,S1,S2,n_round,Z,n_traces)
round_secondpart_phase(S1,S2,n_traces,T1,T2)
TT1, TT2 = round_firstpart_phase(0,SS1,SS2,n_round,Z,n_traces)
round_secondpart_phase(SS1,SS2,n_traces,TT1,TT2)

#SECOND ROUND
n_round =1
# computation of the state after the first part of the permutation
T1, T2 = round_firstpart_phase(1,S1,S2,n_round,Z,n_traces)
# computation of the state after the second part of the permutation
round_secondpart_phase(S1,S2,n_traces,T1,T2)

# correlation computation

bit_mask_whole = ['0b1111111111111111111111111111111111111111111111111111111111111111',
                  '0b1111111111111111111111111111111111111111111111111111111111111111',
                  '0b1111111111111111111111111111111111111111111111111111111111111111',
                  '0b1111111111111111111111111111111111111111111111111111111111111111',
                  '0b1111111111111111111111111111111111111111111111111111111111111111']


H_S1_ab_whole = compute_H_matrix(SS1, bit_mask_whole, 'HD',n_traces, S1)
H_S2_ab_whole = compute_H_matrix(SS2, bit_mask_whole, 'HD',n_traces, S2) 
H_conc_ab_whole = [a + b for a, b in zip(H_S1_ab_whole, H_S2_ab_whole)]

H_S1_input = compute_H_matrix(SS1, bit_mask_whole, 'HW', n_traces) 
H_S2_input = compute_H_matrix(SS2, bit_mask_whole, 'HW', n_traces) 
H_conc_input = [a + b for a, b in zip(H_S1_input, H_S2_input)]

H_S1_output = compute_H_matrix(S1, bit_mask_whole, 'HW', n_traces) 
H_S2_output = compute_H_matrix(S2, bit_mask_whole, 'HW', n_traces) 
H_conc_output = [a + b for a, b in zip(H_S1_output, H_S2_output)]

In [38]:
t = np.transpose(T)
corr_HD = my_correlation(H_conc_ab_whole, t)
corr_HWinput = my_correlation(H_conc_input, t)
corr_HWoutput = my_correlation(H_conc_output, t)

output_notebook()
p = figure(title="Correlation of the traces with HD or HW of all the bits of S1||S2, second round")
p.xaxis.axis_label = "Samples"
p.yaxis.axis_label = "Correlation"
xlen =  len(corr_HD)
xrange = list(range(xlen))
p.line(xrange, corr_HD, line_color=colors[0], legend_label="HD input/output second round")   
p.line(xrange, corr_HWinput, line_color=colors[1], legend_label="HW input second round")  
p.line(xrange, corr_HWoutput, line_color=colors[2], legend_label="HW ouptut second round") 
add_mean_grid_fig_initialization(p, -0.6, 0.30)

show(p)

/Users/MC/Documents/ORSHIN - Ascon analysis/venv/lib/python3.10/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/Users/MC/Documents/ORSHIN - Ascon analysis/venv/lib/python3.10/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


Loading BokehJS ...

## HD or HW of all the bits of S1||S2, last round

In [39]:
# computation of the initial state
S1, S2 = initial_states_computation(IV2, IV1, K2, K1, n_traces)
# INITIAL STATE COMPOSITION: [IV' (64 bits) | Key (128 bits) | IV (128 bits)]

# FIRST 11 ROUNDS
for r in range(11):
    T1, T2 = round_firstpart_phase(r,S1,S2, r ,Z, n_traces)
    round_secondpart_phase(S1,S2,n_traces,T1,T2)

# computation of the initial state
SS1, SS2 = initial_states_computation(IV2, IV1, K2, K1, n_traces)
# INITIAL STATE COMPOSITION: [IV' (64 bits) | Key (128 bits) | IV (128 bits)]

# FIRST 12 ROUNDS
for r in range(12):
    TT1, TT2 = round_firstpart_phase(r,SS1,SS2, r, Z, n_traces)
    round_secondpart_phase(SS1,SS2,n_traces,TT1,TT2)

# correlation computation

bit_mask_whole = ['0b1111111111111111111111111111111111111111111111111111111111111111',
                  '0b1111111111111111111111111111111111111111111111111111111111111111',
                  '0b1111111111111111111111111111111111111111111111111111111111111111',
                  '0b1111111111111111111111111111111111111111111111111111111111111111',
                  '0b1111111111111111111111111111111111111111111111111111111111111111']

# whole round
H_S1_ab_whole = compute_H_matrix(SS1, bit_mask_whole, 'HD',n_traces, S1)
H_S2_ab_whole = compute_H_matrix(SS2, bit_mask_whole, 'HD',n_traces, S2) 
H_conc_ab_whole = [a + b for a, b in zip(H_S1_ab_whole, H_S2_ab_whole)]

H_S1_input = compute_H_matrix(SS1, bit_mask_whole, 'HW', n_traces) 
H_S2_input = compute_H_matrix(SS2, bit_mask_whole, 'HW', n_traces) 
H_conc_input = [a + b for a, b in zip(H_S1_input, H_S2_input)]

H_S1_output = compute_H_matrix(S1, bit_mask_whole, 'HW', n_traces) 
H_S2_output = compute_H_matrix(S2, bit_mask_whole, 'HW', n_traces) 
H_conc_output = [a + b for a, b in zip(H_S1_output, H_S2_output)]

In [41]:
# S1||S2 all the bits of the states
t = np.transpose(T)
corr_HD = my_correlation(H_conc_ab_whole, t)
corr_HWinput = my_correlation(H_conc_input, t)
corr_HWoutput = my_correlation(H_conc_output, t)

output_notebook()
p = figure(title="Correlation of the traces with HD or HW of all the bits of S1||S2, second round")
p.xaxis.axis_label = "Samples"
p.yaxis.axis_label = "Correlation"
xlen =  len(corr_HD)
xrange = list(range(xlen))
p.line(xrange, corr_HD, line_color=colors[0], legend_label="HD input/output second round")   
p.line(xrange, corr_HWinput, line_color=colors[1], legend_label="HW input second round")  
p.line(xrange, corr_HWoutput, line_color=colors[2], legend_label="HW ouptut second round") 
add_mean_grid_fig_initialization(p, -0.6, 0.30)

show(p)

/Users/MC/Documents/ORSHIN - Ascon analysis/venv/lib/python3.10/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/Users/MC/Documents/ORSHIN - Ascon analysis/venv/lib/python3.10/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


Loading BokehJS ...